In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

import inertial_sensor_routines as isr

plt.style.use('ggplot')

In [4]:
path = 'TestACH_20200305_PKMAS_sync3pass.txt'

pkm = pd.read_csv(path, delimiter=';', skiprows=11)

ts = 'Time (sec.)'
pksum = np.zeros(pkm[ts].unique().size)

for i, t in enumerate(pkm[ts].unique()):
    mask = pkm[ts] == t
    
    pksum[i] = pkm.loc[mask, 'Level'].sum()
    
ff = pd.read_csv('TestACH_20200303_PKMAS_Footfalls.csv', skiprows=10, header=None, names=['Sample', 'Pass:side', 'First Contact', 'Last Contact'])

ff['Side'] = ff['Pass:side'].apply(lambda i: i.split(' ')[1])

In [5]:
df = isr.gnactv.tabular_conversion('TestACH20200305-PKMA_back_048877_2020-03-05 14-40-44.bin')

time = df.index.values.astype(int) / 1e9
macc = np.linalg.norm(df[['X', 'Y', 'Z']].values, axis=1)

In [6]:
df.to_csv('gnactv_data.csv')

In [7]:
ff.head(5)

,Sample,Pass:side,First Contact,Last Contact,Side
0,1,1: Right 1,38.942,39.725,Right
1,2,1: Left 1,39.567,40.317,Left
2,3,1: Right 2,40.175,40.883,Right
3,4,1: Left 2,40.758,41.492,Left
4,5,1: Right 3,41.317,42.025,Right


In [8]:
gp_feat = pd.read_csv('gaitpy_features.csv')

gp_fc = gp_feat['IC'].values / 1e3
gp_lc = gp_feat.FC.values / 1e3

gp_fc_s = gp_fc - time[2420] + pkm[ts][0]
gp_lc_s = gp_lc - time[2420] + pkm[ts][0]

FileNotFoundError: [Errno 2] File gaitpy_features.csv does not exist: 'gaitpy_features.csv'

In [9]:
gn_time = time-time[2420] + pkm[ts][0]

fc_ind = np.array([np.argmin(np.abs(gn_time - i)) for i in ff['First Contact']])
lc_ind = np.array([np.argmin(np.abs(gn_time - i)) for i in ff['Last Contact']])

gp_fc_ind = np.array([np.argmin(np.abs(gn_time - i)) for i in gp_fc_s])
gp_lc_ind = np.array([np.argmin(np.abs(gn_time - i)) for i in gp_lc_s])

In [14]:
plt.close('all')

f, (ax, ax1) = plt.subplots(2, figsize=(15, 6), constrained_layout=True, gridspec_kw={'height_ratios': [3, 1]}, sharex=True)
# f.subplots_adjust(hspace=0)

# pressure sensor sum
ax.plot(pkm[ts].unique(), pksum, alpha=0.5, color='C1')

ax.set_ylabel('Pressure Sensor Sum', color='C1')
ax.tick_params(axis='y', colors='C1')
ax.yaxis.set_label_position('right')

# acceleration magnitude
axx = ax.twinx()
axx.grid(False)

axx.plot(gn_time, macc, color='C0')

axx.set_ylabel('Acceleration Magnitude [g]', color='C0')
axx.tick_params(axis='y', colors='C0')
axx.yaxis.set_label_position('left')
axx.yaxis.tick_left()

ax.yaxis.tick_right()  # has to be set after the axx.yaxis.tick_left() call

# second plot - plotting steps

ax1.set_ylim([0.68, 1.02])
ax1.set_xlim([-40, 90])

for fc, lc, side in ff[['First Contact', 'Last Contact', 'Side']].values:
    y = 0.98 if side == 'Right' else 0.9
    c = 'green' if side == 'Left' else 'red'
    ax1.plot([fc, lc], [y, y], linewidth=8, color=c, solid_capstyle='round')

for i, cont in enumerate(zip(gp_fc_s, gp_lc_s)):
    y = 0.8 if i % 2 == 0 else 0.72
    c = 'blue'
    ax1.plot(cont, [y, y], linewidth=8, color=c, solid_capstyle='round')

ax1.set_yticks([0.76, 0.94])
ax1.set_yticklabels(['GaitPy', 'PKMAS'])

ax1.annotate('Right', xy=(1.01, 0.98), annotation_clip=False, verticalalignment='center', xycoords=('axes fraction', 'data'), color='red')
ax1.annotate('Left', xy=(1.01, 0.90), annotation_clip=False, verticalalignment='center', xycoords=('axes fraction', 'data'), color='green')

ax1.annotate('Side 1', xy=(1.01, 0.80), annotation_clip=False, verticalalignment='center', xycoords=('axes fraction', 'data'), color='blue')
ax1.annotate('Side 2', xy=(1.01, 0.72), annotation_clip=False, verticalalignment='center', xycoords=('axes fraction', 'data'), color='blue')

ax1.set_xlabel('Time [s]')

# spike inset
axins = ax.inset_axes([0.03, 0.4, 0.27, 0.57])

axins.plot(pkm[ts].unique(), pksum, color='C1', alpha=0.5)
axins.plot(gn_time, macc * (100 / 12.2), color='C0')

axins.set_xlim([4.0, 5.5])
axins.set_ylim([-0.35, 100])
axins.tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)

ax.indicate_inset_zoom(axins)

# first pass steps inset

axins1 = ax1.inset_axes([0.03, 0.03, 0.3, 0.94])

for fc, lc, side in ff[['First Contact', 'Last Contact', 'Side']].values:
    y = 0.98 if side == 'Right' else 0.9
    c = 'green' if side == 'Left' else 'red'
    axins1.plot([fc, lc], [y, y], linewidth=8, color=c, solid_capstyle='round')

for i, cont in enumerate(zip(gp_fc_s, gp_lc_s)):
    y = 0.8 if i % 2 == 0 else 0.72
    c = 'blue'
    axins1.plot(cont, [y, y], linewidth=8, color=c, solid_capstyle='round')
    
axins1.set_xlim([38.75, 43.5])
axins1.tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
ax1.indicate_inset_zoom(axins1)

# f.savefig('alignment_visualization.png')
# f.savefig('alignment_visualization.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.patches.Rectangle at 0x7fc3f8b35940>,
 (<matplotlib.patches.ConnectionPatch at 0x7fc3f8b89fa0>,
  <matplotlib.patches.ConnectionPatch at 0x7fc3f8b94880>))

In [81]:
3 % 2

1

In [ ]:
np.argmax(macc)

In [ ]:
path = 'data/Test2_ACH_20200303_GaitRaw.txt'

df = pd.read_csv(path, names=['Time', 'x', 'y', 'F'], header=None)

In [ ]:
path = 'data/Test2ACH20200303_back.csv'
acc_df = pd.read_csv(path, names=['Time', 'x', 'y', 'z', '1', '2', '3'], header=None, skiprows=100)

acc_df.head(5)

mag = np.linalg.norm(acc_df[['x', 'y', 'z']].values, axis=1)
timestamps = pd.to_datetime(acc_df.Time, format='%Y-%m-%d %H:%M:%S:%f').astype(int) / 1e9

timestamps -= timestamps[0]

In [ ]:
allsum = np.zeros(df.Time.unique().size)

for i, t in enumerate(df.Time.unique()):
    mask = df.Time == t
    allsum[i] = df.loc[mask, 'F'].sum()

In [ ]:
f, ax = plt.subplots(figsize=(12, 6), constrained_layout=True)

ax.plot(df.Time.unique(), allsum)
axx = ax.twinx()
axx.grid(False)
axx.plot(timestamps - 18.375, mag, color='C1')

In [ ]:
detail = pd.read_csv('data/Test2_ACH_20200303_FootDetail.csv', skiprows=1, nrows=7)

foot = detail['Foot ']
ic = detail['FirstContact ']
fc = detail['LastContact ']

for s, e, f in zip(ic, fc, foot):
    c = 'C2' if f else 'C3'
    ax.axvspan(s, e, color=c, alpha=0.3)

In [ ]:
path = 'data/Test_ACH_doubleDrop.txt'
ddf = pd.read_csv(path, names=['Time', 'x', 'y', 'F'], header=None)

In [ ]:
asum = np.zeros(ddf.Time.unique().size)
for i, t in enumerate(ddf.Time.unique()):
    mask = ddf.Time == t
    mask &= ddf.x > 435
    asum[i] = ddf.loc[mask, 'F'].sum()

plt.figure()
plt.plot(asum)

In [ ]:
from matplotlib.animation import FuncAnimation

In [ ]:
ut = ddf.Time.unique()
x = ddf.loc[:, 'x'].values
y = ddf['y'].values
F = ddf['F'].values

# f, ax = plt.subplots(5, 5, sharex=True, sharey=True, figsize=(12, 12))

# cnt = 0
# for i in range(5):
#     for j in range(5):
#         mask = ddf.Time == ut[cnt]
#         X, Y, f2d = get_grids(x[mask], y[mask], F[mask])
#         ax[i, j].pcolormesh(X, Y, f2d, cmap='Blues')
        
#         cnt += 30

def get_plot(frame, time, utime, xvals, yvals, fvals, X, Y):
    f2d = Z0
    
    mask = time == utime[frame]
    
    for i, j, f in zip(xvals[mask], yvals[mask], fvals[mask]):
        m = (X == i) & (Y == j)
        f2d[m] += f
    
    quad.set_array(f2d.ravel())
    
    ax.set_title(f'Time: {utime[frame]:.2f}s')
    
    return quad

def init():
#     quad.set_array([])
    ax.set_title('init')
    return quad


X, Y = np.meshgrid(np.arange(x.min(), x.max()), np.arange(y.min(), y.max()))
Z0 = np.zeros_like(X)

mask = ddf.Time.values == ut[0]
for i, j, f in zip(x[mask], y[mask], F[mask]):
    m = (X==i) & (Y==j)
    Z0[m] = f

f, ax = plt.subplots(figsize=(13, 13), constrained_layout=True)
quad = ax.pcolormesh(X, Y, Z0, cmap='Blues', vmin=0, vmax=F.max())

anim = FuncAnimation(f, get_plot, init_func=init, frames=ut.size, interval=20, blit=True, repeat=False, fargs=(ddf.Time.values, ut, x, y, F, X, Y))
# plt.show()

In [ ]:
plt.close('all')